Copyright (c) 2023 Ivan Magistro Contenta, Matteo Pietro Pillitteri and Francesca Zafonte (<s314356@studenti.polito.it> , <s314404@studenti.polito.it>, <s319331@studenti.polito.it>)
https://github.com/ivanmag22/computational-intelligence
https://github.com/Matteo-Pietro-Pillitteri/Computational-Intelligence
https://github.com/Zafonte/computational-intelligence

In [58]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

**General case:**

In [59]:
PROBLEM_SIZE = 20  # elements to cover
NUM_SETS = 12  # tails in SET.
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)
State = namedtuple("State", ["taken", "not_taken"])

**Goal**: minimize the number of sets (num of array in SETS used to cover all the elements)

In [60]:
SETS

(array([False, False, False, False, False, False, False,  True, False,
        False,  True, False, False,  True, False, False, False, False,
        False, False]),
 array([ True,  True,  True, False, False, False, False, False, False,
        False, False, False, False, False,  True,  True,  True, False,
        False,  True]),
 array([False,  True,  True, False, False, False, False, False, False,
        False,  True, False,  True,  True, False, False, False,  True,
         True, False]),
 array([ True, False,  True, False,  True, False, False, False,  True,
        False, False, False,  True, False,  True,  True, False, False,
        False,  True]),
 array([False,  True, False, False,  True,  True, False,  True, False,
        False,  True,  True, False, False,  True, False,  True, False,
        False, False]),
 array([False, False, False, False,  True, False, False, False,  True,
        False,  True, False, False,  True,  True, False, False, False,
        False,  True]),
 arr

**Specific case**: we created **SETS** (manually) that shows that the algorithm chooses the state with the lowest overlap if there are states with the same cost (**number of tiles  + distance to the goal**)

In [61]:
tmp = tuple(
    np.array(
        [
            [True, True, False, False, True],
            [True, False, False, True, False],
            [True, True, True, False, False],
            [False, False, False, False, False],
            [False, False, True, False, False],
        ]
    )
)
State = namedtuple("State", ["taken", "not_taken"])

In [62]:
tmp

(array([ True,  True, False, False,  True]),
 array([ True, False, False,  True, False]),
 array([ True,  True,  True, False, False]),
 array([False, False, False, False, False]),
 array([False, False,  True, False, False]))

It is possible to use the following code for both cases 

In [63]:
def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

def covered(state):
    return reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )

def heuristic(
    state,
):  # this function gives the distance to the goal state in terms of number of missing Trues
    return PROBLEM_SIZE - np.sum(covered(state))


def overlap_cost(
    state,
):  # this function returns the cost due to overlapping among the array in SETS
    set_overlap = np.array(
        np.sum([SETS[i] for i in state.taken], axis=0)
    )
    res = [x - 1 if x > 1 else 0 for x in set_overlap]
    return np.sum(res) / (PROBLEM_SIZE * len(state.taken))


def g(state):
    return len(state.taken) + overlap_cost(state)

def f_cost(state):    # f(n) = h(n) + g(n)
    print("heuristic:", heuristic(state))
    return heuristic(state) + g(state)

We know that our heuristic cost function is pessimistic because each time our function returns the distance d from the current state to the goal one. So it means that we need d sets to reach the goal state, but it is possible to reach it also with less sets.

A possible heuristic cost is the one that returns the minimum number of sets that could cover the whole interval of elements (as implemented by prof. Squillero in set-covering_path-search.ipynb).

In [64]:
def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

def covered(state):
    return reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )

def h3(state):  # given the state, we compute the heuristic cost as the minimum number of needed sets to add to the current taken sets that could cover the interval  
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(s, np.logical_not(already_covered))) for s in SETS), reverse=True)
    #print("\th3(): print candidates of ",state.taken,": ",candidates)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    #print("\th3(): print heuristic cost of ",state.taken,": ",taken)
    return taken


def overlap_cost(
    state,
):  # this function returns the cost due to overlapping among the array in SETS
    if(len(state.taken)>1): # we put a condition on the number of taken sets in order to avoid to work with 0 dimensional array and to do not work with only one vector because there is no overlap in this case
        set_overlap = np.array(
            np.sum([SETS[i] for i in state.taken], axis=0)
        )
        res = [x - 1 if x > 1 else 0 for x in set_overlap]
        return np.sum(res) / (PROBLEM_SIZE * len(state.taken))
    else:
        return 0


def g(state):
    return len(state.taken) + overlap_cost(state)

def f_cost(state):  # f(n) = h(n) + g(n)
    #print("heuristic: ", h3(state))
    return h3(state) + g(state)

In [65]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Problem not solvable"

frontier elements: index 0 -> cost ; index 1 -> state

In [66]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f_cost(state), state))

counter = 0
current_cost, current_state = frontier.get()
while not goal_check(current_state):
    print("current state: ", current_state,"; current cost: ",current_cost)
    counter += 1
    for action in current_state.not_taken:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f_cost(new_state), new_state))
    current_cost, current_state = frontier.get()
print("solution: ", current_state,"; current cost: ", current_cost, "\n")

print(
    f"Solved in {counter:,} steps, tiles: ({len(current_state.taken)})"
)

current state:  State(taken=set(), not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}) ; current cost:  3
current state:  State(taken={5}, not_taken={0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11}) ; current cost:  3
current state:  State(taken={6}, not_taken={0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11}) ; current cost:  3
current state:  State(taken={11}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}) ; current cost:  3
current state:  State(taken={7}, not_taken={0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11}) ; current cost:  3
current state:  State(taken={9}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11}) ; current cost:  3
current state:  State(taken={0}, not_taken={1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}) ; current cost:  4
current state:  State(taken={2}, not_taken={0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11}) ; current cost:  4
current state:  State(taken={10}, not_taken={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11}) ; current cost:  4
current state:  State(taken={4}, not_taken={0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11}) ; current cost:  4
curre

In [67]:
overlap_cost(current_state)

0.1

In [68]:
current_state

State(taken={8, 10, 4, 6}, not_taken={0, 1, 2, 3, 5, 7, 9, 11})

The following code tests the PriorityQueue when there are both decimal and integer costs. It is possible to use a decimal cost for assign less importance to a state with an higher overlap. So, this decimal contribution to the overall cost of a state, is due to **overlapping** and it moves from 0.1 to 0.9 according to the number of overlapped elements. In order to prioritize the state with a minimum overlap, it's possbile to assign a lower decimal value. 

In [69]:
prova = PriorityQueue()
prova.put((2.2, "ciao"))
prova.put((2, "ivan"))
prova.put((3, "matteo"))

In [70]:
prova.get()

(2, 'ivan')

The following code shows how it's possible to calculate the overlap

In [71]:
PROVA_SETS = tuple(
    np.array([random() < 0.6 for _ in range(PROBLEM_SIZE)])
    for _ in range(2)
)



In [72]:
PROVA_SETS

(array([ True, False, False,  True,  True, False,  True,  True, False,
         True,  True,  True,  True,  True,  True, False,  True,  True,
        False,  True]),
 array([ True,  True, False, False, False, False,  True, False, False,
         True, False, False,  True, False, False,  True,  True,  True,
         True, False]))

In [73]:
res = np.array(np.sum(PROVA_SETS, axis=0))
res

array([2, 1, 0, 1, 1, 0, 2, 1, 0, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1])

If there are 3 arrays, for istance [1, 1, 0, ..]
                                   [1, 0, 0, ..]
                                   [1, 1, 1, ..]
for the first element there is an overlap between the first and the second array. Then, there is another overlap between the second and the third array. In sum, for the first element it's possible to count an overlap equals to 2.

In [74]:
res = [x - 1 if x > 1 else 0 for x in res]

In [75]:
res

[1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0]

In order to calculate a decimal cost it is possible to sum the values of the array evaluated from the previous step and divide it for the total number of elements in the "taken set". In this small example PROBLEM_SIZE * len(PROVA_SETS) represents the taken set

In [76]:
x = np.sum(res) / (PROBLEM_SIZE * len(PROVA_SETS))

In [77]:
x

0.15